# LogisticAT

Este notebook prepara los datos, entrena y evalúa un modelo de regresión ordinal para predecir la variable de rendimiento. El flujo principal incluye: carga y mapeo de etiquetas ordinales, conversión de variables tipo objeto a códigos, escalado de características, partición estratificada en train/test y evaluación con accuracy, macro‑F1, reporte de clasificación y matriz de confusión.

**Modelo utilizado**
- Se usa LogisticAT (módulo mord), una regresión logística ordinal de tipo "all‑threshold" empaquetada en un Pipeline con StandardScaler.

**Motivo de no selección para la entrega final**
- El modelo no se seleccionó porque mostró bajo desempeño en las métricas clave (accuracy y macro‑F1) y alta confusión entre clases adyacentes, incumpliendo los requisitos de calidad esperados.
- Posibles causas: la relación ordinal no está bien modelada por este enfoque lineal, características insuficientes o necesidad de mayor ingeniería de variables y balanceo de clases.
- Recomendación: probar modelos alternativos (p. ej. modelos basados en árboles o boosting con tratamiento ordinal), optimizar features y reequilibrar clases antes de la entrega final.

In [ ]:
import os
import time
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from mord import LogisticAT   # modelo ordinal
RND = 42 # Se usa una semilla fija para reproducibilidad

### 2. Carga de los archivos para el entrenamiento

Para esto se realiza lo siguiente:

- Se cargan los archivos con pandas
- Se extraen los labels de `RENDIMIENTO GLOBAL` y se transforman en numeros ordinale

In [ ]:
X_path = "data/X_preprocessed.csv"
y_path = "data/y_preprocessed.csv"

X = pd.read_csv(X_path)
y_df = pd.read_csv(y_path)

if 'RENDIMIENTO_GLOBAL' in y_df.columns:
    y_raw = y_df['RENDIMIENTO_GLOBAL']


# Mapping ordinal — ajusta si tus nombres cambian
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}
# Aplicar map (si aparecen valores no mapeados, mostramoslos)
y_mapped = y_raw.map(label_map)
if y_mapped.isna().any():
    missing_vals = sorted(set(y_raw[y_mapped.isna()]))
    raise RuntimeError(f"Hay etiquetas en y que no están en label_map: {missing_vals}")

y = y_mapped.astype(int).to_numpy()

print("X shape:", X.shape)
print("y distribution:\n", pd.Series(y).value_counts().sort_index())

X shape: (690861, 17)
y distribution:
 0    172561
1    171857
2    171224
3    175219
Name: count, dtype: int64


### 3. Ensurement respecto a la estructura de los datos de las columnas y recuento de NAN

In [ ]:
# Convertir columnas object a códigos (int). Conserva NaN si existen.
obj_cols = X.select_dtypes(include=['object']).columns.tolist()
for c in obj_cols:
    X[c] = X[c].astype('category').cat.codes  # -1 para NaN (cat.codes devuelve -1 para NaN)

# Revisar tipos y NaNs
print("dtypes sample:\n", X.dtypes.value_counts())
print("NA counts (head):\n", X.isna().sum().sort_values(ascending=False).head(10))

dtypes sample:
 int64      10
float64     7
Name: count, dtype: int64
NA counts (head):
 PERIODO_ACADEMICO              0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
dtype: int64


### 4. Sampleo (para test anteriores y separacion de los datos)

- Define `SAMPLE_N` para usar todo el dataset o hacer un submuestreo rápido. Si `SAMPLE_N` es un entero menor que el total, realiza un muestreo estratificado para mantener proporciones de clase y reinicia índices; si no, solo reinicia índices y asegura que `y` sea un array numpy. Imprime el tamaño usa

- Divide los datos en train/test con `test_size=0.10` y estratificación por `y`. Reinicia índices de los DataFrames, convierte `y` a numpy arrays y muestra formas de los conjuntos y la distribución de clases en train y test.

por ultimo se crea un `Pipeline` con `StandardScaler` y `mord.LogisticAT` (regresión logística ordinal), ajusta el pipeline sobre `X_train, y_train`, mide el tiempo de entrenamiento y muestra la duración.

In [ ]:
# Si tiene muchas filas y quiere probar rápido, activar sub-sampling.
# Poner SAMPLE_N = None para usar todo el dataset.
SAMPLE_N = None   # e.g. 50000  or None

if SAMPLE_N is not None and SAMPLE_N < X.shape[0]:
    # estratified sample to keep class proportions
    X_tmp, _, y_tmp, _ = train_test_split(X, y, train_size=SAMPLE_N, stratify=y, random_state=RND)
    X = X_tmp.reset_index(drop=True)
    y = y_tmp
    print(f"Subsampled to {len(y)} rows")
else:
    X = X.reset_index(drop=True)
    y = np.asarray(y)
    print("Using full dataset:", X.shape[0], "rows")


Using full dataset: 690861 rows


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, stratify=y, random_state=RND
)

# Reset index safety
X_train = X_train.reset_index(drop=True)
X_test  = X_test.reset_index(drop=True)
y_train = np.asarray(y_train)
y_test  = np.asarray(y_test)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Train class dist:", pd.Series(y_train).value_counts().sort_index().to_dict())
print("Test class dist:", pd.Series(y_test).value_counts().sort_index().to_dict())


Train shape: (621774, 17) Test shape: (69087, 17)
Train class dist: {0: 155305, 1: 154671, 2: 154101, 3: 157697}
Test class dist: {0: 17256, 1: 17186, 2: 17123, 3: 17522}


In [18]:
# Mord works with numpy arrays; use a pipeline for scaling numeric features
pipe = Pipeline([
    ("scaler", StandardScaler()),      # escalar numéricos
    ("ordclf", LogisticAT())           # modelo ordinal
])

t0 = time.time()
pipe.fit(X_train, y_train)
t1 = time.time()
print(f"Trained LogisticAT ordinal model in {t1-t0:.1f} sec")


Trained LogisticAT ordinal model in 18.7 sec


El tiempo de entrenamiento de este modelo es especialmente rapido a pesar de haber sido entrenado con el 90% del dataset de entrenamiento (621774 datos)

In [19]:
y_pred = pipe.predict(X_test)

acc = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Test accuracy:", acc)
print("Test macro-F1:", macro_f1)
print("\nClassification report:\n", classification_report(y_test, y_pred, digits=4))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))


Test accuracy: 0.3642798210951409
Test macro-F1: 0.3611642145381146

Classification report:
               precision    recall  f1-score   support

           0     0.5484    0.2057    0.2992     17256
           1     0.3058    0.5139    0.3835     17186
           2     0.2940    0.4426    0.3533     17123
           3     0.6543    0.2972    0.4088     17522

    accuracy                         0.3643     69087
   macro avg     0.4506    0.3648    0.3612     69087
weighted avg     0.4519    0.3643    0.3613     69087


Confusion matrix:
 [[3549 9515 3872  320]
 [1904 8832 5703  747]
 [ 783 7077 7578 1685]
 [ 235 3455 8624 5208]]


### Interpretación de los datos
- Las métricas finales (accuracy y macro‑F1) muestran un desempeño limitado: el modelo no discrimina bien todas las clases ordinales.  
- El reporte de clasificación y la matriz de confusión indican mucha confusión entre clases adyacentes (p. ej. "medio‑bajo" ↔ "medio‑alto"), típico cuando la separación lineal es insuficiente.  
- Tiempo de entrenamiento cómodo, pero la capacidad predictiva es la variable crítica.

### Conclusión final
El pipeline con LogisticAT no es adecuado para la entrega final dado su bajo desempeño y la alta confusión entre clases.